In [1]:
import pandas as pd

dataset = pd.read_csv(r"C:\Users\AVTSLU744\Downloads\IMDB Dataset.csv")

def convert_label(inp):
    return 0 if inp == "negative" else 1
  
dataset["sentiment"] = dataset["sentiment"].apply(lambda x: convert_label(x))

train_set = dataset[0:40]
valid_set = dataset[40:45]
test_set  = dataset[45:50]

# train_set.head()

train_set = train_set.drop("sentiment", axis=1)
valid_set = valid_set.drop("sentiment", axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\AVTSLU744\\Downloads\\IMDB Dataset.csv'

In [9]:
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AVTSLU744\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    
    if scores["neg"] > scores["pos"]:
        return 0

    return 1

train_set["vader_result"] = train_set["review"].apply(lambda x: vader_sentiment_result(x))
valid_set["vader_result"] = valid_set["review"].apply(lambda x: vader_sentiment_result(x))

In [13]:
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = self.max_len,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_set_dataset = TheDataset(
    reviews    = train_set.review.tolist(),
    sentiments = train_set.vader_result.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    reviews    = valid_set.review.tolist(),
    sentiments = valid_set.vader_result.tolist(),
    tokenizer  = tokenizer,
)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# Freeze BERT except (the 24th layer + the last pooler layer)
for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.23" not in name :
        param.requires_grad = False

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = "./sentiment-analysis",
    num_train_epochs            = 5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 64,
    warmup_steps                = 500,
    weight_decay                = 0.01,
    save_strategy               = "epoch",
    evaluation_strategy         = "steps"
)

# Define the Huggingface Trainer object
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics
)

# Start pre-training!
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\AVTSLU744/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at 

Step,Training Loss,Validation Loss


Saving model checkpoint to ./sentiment-analysis\checkpoint-3
Configuration saved in ./sentiment-analysis\checkpoint-3\config.json
Model weights saved in ./sentiment-analysis\checkpoint-3\pytorch_model.bin
Saving model checkpoint to ./sentiment-analysis\checkpoint-6
Configuration saved in ./sentiment-analysis\checkpoint-6\config.json
Model weights saved in ./sentiment-analysis\checkpoint-6\pytorch_model.bin
Saving model checkpoint to ./sentiment-analysis\checkpoint-9
Configuration saved in ./sentiment-analysis\checkpoint-9\config.json
Model weights saved in ./sentiment-analysis\checkpoint-9\pytorch_model.bin
Saving model checkpoint to ./sentiment-analysis\checkpoint-12
Configuration saved in ./sentiment-analysis\checkpoint-12\config.json
Model weights saved in ./sentiment-analysis\checkpoint-12\pytorch_model.bin
Saving model checkpoint to ./sentiment-analysis\checkpoint-15
Configuration saved in ./sentiment-analysis\checkpoint-15\config.json
Model weights saved in ./sentiment-analysis\c

TrainOutput(global_step=15, training_loss=1.0915589650472006, metrics={'train_runtime': 994.2829, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.015, 'total_flos': 186386272665600.0, 'train_loss': 1.0915589650472006, 'epoch': 5.0})

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    
    if scores["neg"] > scores["pos"]:
        return 0

    return 1

train_set["vader_result"] = train_set["review"].apply(lambda x: vader_sentiment_result(x))
valid_set["vader_result"] = valid_set["review"].apply(lambda x: vader_sentiment_result(x))

In [15]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("./sentiment-analysis/checkpoint-12")

# Make the test set ready
test_set_dataset = TheDataset(
    reviews    = test_set.review.tolist(),
    sentiments = test_set.sentiment.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = "./sentiment-analysis",
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

trainer.predict(test_set_dataset)

loading configuration file ./sentiment-analysis/checkpoint-12\config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.10.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./sentiment-analysis/checkpoint-12\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the w

C:\Users\AVTSLU744\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 0.5463936 , -0.9056816 ],
       [ 0.5585799 , -0.81527245],
       [ 0.58883065, -0.95091134],
       [ 0.71747327, -0.8232268 ],
       [ 0.75908893, -0.96796966]], dtype=float32), label_ids=array([1, 0, 0, 1, 0], dtype=int64), metrics={'test_loss': 0.7961587309837341, 'test_accuracy': 0.6, 'test_f1': 0.0, 'test_precision': 0.0, 'test_recall': 0.0, 'test_runtime': 20.6822, 'test_samples_per_second': 0.242, 'test_steps_per_second': 0.048})

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    
    if scores["neg"] > scores["pos"]:
        return 0

    return 1
  
test_set.loc[:,("vader_result")] = test_set.loc[:,("review")].apply(lambda x: vader_sentiment_result(x))
pd.options.mode.chained_assignment = None
accuracy = accuracy_score(test_set.loc[:,("sentiment")], test_set.loc[:,("vader_result")])

In [20]:
print(accuracy)

0.6
